In [74]:
import torch
import numpy as np
from tqdm import tqdm
from analytical_expressions import local_energy
from torch.autograd.functional import jacobian
from torch.func import jacrev
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def psi(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [90]:
L = 1
r1 = torch.rand(3, requires_grad=True) * 2 * L - L
r2 = torch.rand(3, requires_grad=True) * 2 * L - L #random number from -L to L
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)
E = 0
E2 = 0
Eln_average = 0
ln_average = 0
rejection_ratio = 0
step = 0
max_steps = 500
N = 10000
dlap_dalpha = 0
inputs_arr = []
n_walkers = 5

for walkers in range(n_walkers):
    inputs = []
    for i in tqdm(range(N)):

        chose = np.random.rand()
        step = step + 1
        if chose < 0.5:
            r1_trial = r1 + 0.5 * (torch.rand(3) * 2 * L-L)
            r2_trial = r2
        else:
            r2_trial = r2 + 0.5 * (torch.rand(3) * 2 * L-L)
            r1_trial = r1

        X = torch.tensor(
            [*r1, *r2, alpha_1, alpha_2, alpha_3, alpha_4], requires_grad=True)
        X_trial = torch.tensor(
            [*r1_trial, *r2_trial, alpha_1, alpha_2, alpha_3, alpha_4], requires_grad=True)

        if psi(X_trial) >= psi(X):
            r1 = r1_trial
            r2 = r2_trial
            
        else:
            dummy = np.random.rand()
            if dummy < psi(X_trial) / psi(X):
                r1 = r1_trial
                r2 = r2_trial
            else:
                rejection_ratio += 1./N
                    
        if step > max_steps:
            
            X_final = torch.tensor(
            [*r1, *r2, alpha_1, alpha_2, alpha_3, alpha_4], requires_grad=True)

            #local_E = local_energy(X_final)
            #E += local_E / (N - max_steps)
            #dlap_dalpha += torch.autograd.grad(local_E, X_final, retain_graph=True)[0][6:] / (N - max_steps)
            inputs.append(X_final)

    inputs_arr.append(inputs)




  5%|▍         | 484/10000 [00:00<00:07, 1196.54it/s]


KeyboardInterrupt: 

In [91]:
rejection_ratio

0.20489999999999375

In [86]:
rejection_ratio

0.21559999999999258

In [11]:
dlap_dalpha

tensor([-0.0002,  0.0014,  0.0011, -0.0007], dtype=torch.float64)

In [80]:
def dE_dalpha(input):
    return jacrev(local_energy)(input)

In [81]:
from torch.func import vmap

local_e_vec = vmap(local_energy)
dE_dalpha_vec = vmap(dE_dalpha)

energies = [local_e_vec(torch.stack(inputs_arr[i])) for i in range(len(inputs_arr))]

In [88]:
sum(energies[0])/(len(energies[0]))

tensor(-18.8979, dtype=torch.float64, grad_fn=<DivBackward0>)

In [69]:
sum([sum(energies[i])/len(energies[i]) for i in range(len(energies))])

tensor(-3.0756, dtype=torch.float64, grad_fn=<AddBackward0>)

In [51]:
t = dE_dalpha_vec(torch.stack(inputs_arr[0]))

In [54]:
t[:, 6:10]

tensor([[ 5.6105e-04, -6.9556e-03, -1.3911e-03,  2.1733e-03],
        [ 1.9731e-04, -2.9057e-03, -7.3384e-04,  9.8745e-04],
        [ 1.9731e-04, -2.9057e-03, -7.3384e-04,  9.8745e-04],
        ...,
        [ 1.2597e-05,  1.0297e-03, -8.4545e-04,  8.5725e-04],
        [ 1.0823e-04,  3.0629e-04, -7.4649e-04,  5.3556e-04],
        [ 3.3389e-04, -7.8640e-04, -1.5131e-03,  7.2224e-04]],
       dtype=torch.float64, grad_fn=<SliceBackward0>)

In [22]:
len(inputs_arr[0])

49500